In [127]:
# Load the data into a pandas dataframe:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [128]:
# roomifAI_dir = '/content/drive/MyDrive/MTECH_IS_Documents/PracticeModule/roomifAI'
roomifAI_dir = '/content/drive/MyDrive/MTECH_IS_Project/SEM02'



In [129]:
!pip install gitpython

In [130]:
# importing libary
import zipfile
import os
import shutil
import requests
import git
import re
import pandas as pd
import logging

In [139]:
# Define the git repo url and the destination directory
git_repo_url = "https://github.com/valexande/IKEA-Dataset.git" # Change this to your git repo url
dest_dir = roomifAI_dir + "/dataset" # Change this to your destination directory
log_dir = roomifAI_dir + "/logs" # Change this to your prepared log directory
temp_dir = roomifAI_dir + "/temp/ikea_dir" # Change this to your temporary directory

# Create Directory if don't exist
os.makedirs(dest_dir, exist_ok=True)
print(dest_dir)


# Configure logging
log_file = log_dir + '/get_dataset.log'
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')

# Create Directory if don't exist
os.makedirs(log_dir, exist_ok=True)
print(log_dir)


/content/drive/MyDrive/MTECH_IS_Project/SEM02/dataset
/content/drive/MyDrive/MTECH_IS_Project/SEM02/logs


In [132]:
# Define a function that will download and unzip a git repo
def download_git_repo(url, dest):
    # Create a temporary directory to store the downloaded zip file
    temp_ikea_dir = os.path.abspath(temp_dir)

    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    elif not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    elif not os.path.exists(log_dir):
        os.makedirs(log_dir)
    else:
        # shutil.rmtree(temp_dir) # Delete the temporary directory and its contents
        # shutil.rmtree(dest_dir) # Delete the temporary directory and its contents
        os.makedirs(temp_dir) # Re-creating a empty directory
        os.makedirs(dest_dir) # Re-creating a empty directory

    print(dest_dir)
    file_name = url.split("/")[-1] # Get the file name from the url
    zip_path = os.path.join(temp_dir, file_name) # Get the full path of the zip file
    response = git.Git(temp_dir).clone(url) # Download the git repo  from the url

    return(temp_ikea_dir)

In [133]:
# Define the function that will unzip the zip files
def unzip_ikeads(url, get_temp_ikea_dir):
  temp_dir = get_temp_ikea_dir # Get the value of the temp directory from previous function
  file_name = url.split("/")[-1] # Get the file name from the url
  print(file_name)
  zip_path = os.path.join(temp_dir, file_name) # Get the full path of the zip file
  print(zip_path)
  # print (zip_path) # this include ".git" ikeatemp/IKEA-Dataset.git
  temp_ds_path = zip_path.split(".")[0] # spliting to get the relative path and directory name
  print(temp_ds_path)
  # Get the list of files in the source directory
  files = os.listdir(temp_ds_path)
  print(files)

  # Loop through the files and unzip them if they are zip files
  for file in files:
    if file.endswith(".zip"): # Check if the file is a zip file
      file_path = os.path.join(temp_ds_path, file) # Get the full path of the file
      zip_file = zipfile.ZipFile(file_path) # Create a ZipFile object
      zip_file.extractall(temp_dir) # Extract all the contents to the destination directory
      list_des = os.listdir(temp_dir)
      zip_file.close() # Close the ZipFile object

  return temp_dir


In [134]:
# Function to get all jpg file paths in a directory and its subdirectories
def list_jpg_files(get_temp_ikea_dir):
    jpg_files = []
    for root, dirs, files in os.walk(get_temp_ikea_dir):
        for file in files:
            if file.lower().endswith(".jpg"):
                jpg_files.append(os.path.join(root, file))
    return jpg_files # This is the list to call when you want to ge the list of path


# Function that will put the list of path and add them in a dataframe and process them
def df_jpg_files(get_temp_ikea_dir):
    # Get the list of jpg file paths
    jpg_file_paths = list_jpg_files(get_temp_ikea_dir)

    # Create a DataFrame and process the paths
    jpg_df = pd.DataFrame({'FilePath': jpg_file_paths})

    # Replace backslashes with forward slashes, convert to lowercase, and replace spaces with underscores
    jpg_df['FilePath'] = jpg_df['FilePath'].apply(lambda x: x.replace("/", ",").lower().replace(" ", "_"))

    # Split the 'FilePath' column into separate columns using a comma as delimiter
    jpg_df['Columns'] = jpg_df['FilePath'].str.split(',')

    # Determine the maximum number of path parts across all rows
    max_path_parts = jpg_df['Columns'].apply(len).max()

    # Create separate columns for each path part
    for i in range(max_path_parts):
      jpg_df[f'Column_{i+1}'] = jpg_df['Columns'].apply(lambda x: x[i] if len(x) > i else None)

    # Drop the intermediate 'PathParts' column
    jpg_df = jpg_df.drop(columns=['Columns','FilePath','Column_1','Column_2','Column_3','Column_4','Column_5','Column_6','Column_7','Column_8'])

    # print(jpg_df)
    return jpg_file_paths

# Function to get all txt file paths in a directory and its subdirectories
def list_txt_files(get_temp_ikea_dir):
    txt_files = []
    for root, dirs, files in os.walk(get_temp_ikea_dir):
        for file in files:
            if file.lower().endswith(".txt"):
                txt_files.append(os.path.join(root, file))
    return txt_files # This is the list to call when you want to ge the list of path

# Function that will put the list of path and add them in a dataframe and process them
def df_txt_files(get_temp_ikea_dir):
    # Get the list of jpg file paths
    txt_file_paths = list_txt_files(get_temp_ikea_dir)

    # Create a DataFrame and process the paths
    txt_df = pd.DataFrame({'FilePath': txt_file_paths})

    # Replace backslashes with forward slashes, convert to lowercase, and replace spaces with underscores
    txt_df['FilePath'] = txt_df['FilePath'].apply(lambda x: x.replace("/", ",").lower().replace(" ", "_"))

    # # Split the 'FilePath' column into separate columns using a comma as delimiter
    # txt_df[['Column1', 'Column2', 'Column3', 'Column4', 'Column5']] = txt_df['FilePath'].str.split('/', expand=True)

    # Split the 'FilePath' column into separate columns using a comma as delimiter
    txt_df['Columns'] = txt_df['FilePath'].str.split(',')

    # Determine the maximum number of path parts across all rows
    max_path_parts = txt_df['Columns'].apply(len).max()

    # Create separate columns for each path part
    for i in range(max_path_parts):
        txt_df[f'Column_{i+1}'] = txt_df['Columns'].apply(lambda x: x[i] if len(x) > i else None)

    # Drop the intermediate 'PathParts' column
    txt_df = txt_df.drop(columns=['Columns','FilePath','Column_1','Column_2','Column_3','Column_4','Column_5','Column_6','Column_7','Column_8'])

    # print(txt_df)
    return txt_file_paths

In [135]:
# Function to move jpg files based on the source file path to a centralize location

# Function to move files based on the source file path
def move_files(jpg_path_list, dest_dir):

    # Destination Create the directory where all consolidated jpg files be located
    destination_dir = dest_dir + "/ikeads_all_images"

    # Create Directory if don't exist
    os.makedirs(destination_dir, exist_ok=True)
    print(destination_dir)

    # Create a DataFrame and process the paths
    source_df = pd.DataFrame({'FilePath': jpg_path_list})

    for index, row in source_df.iterrows():
        source_file = row['FilePath']
        if not os.path.exists(source_file):
            logging.error(f"Source file not found: {source_file}")
            continue

        destination_file = os.path.join(destination_dir, os.path.basename(source_file))
        try:
            shutil.move(source_file, destination_file)
            logging.info(f"Moved file: {source_file} to {destination_file}")
        except Exception as e:
            logging.error(f"Error moving file: {source_file} - {str(e)}")

    return destination_dir


In [140]:
# Call the first function with the git repo url and destination directory as arguments

if os.path.exists(temp_dir):
  shutil.rmtree(temp_dir) # Delete the previous run temporary directory and its contents

get_temp_ikea_dir = download_git_repo(git_repo_url, dest_dir)

# # Call function to unzip
unzip_ikea_files = unzip_ikeads(git_repo_url, get_temp_ikea_dir)

# Call Function which will get the jpg and txt and
jpg_path_list = df_jpg_files(unzip_ikea_files)
txt_path_list = df_txt_files(unzip_ikea_files)

print(jpg_path_list)


/content/drive/MyDrive/MTECH_IS_Project/SEM02/dataset
IKEA-Dataset.git
/content/drive/MyDrive/MTECH_IS_Project/SEM02/temp/ikea_dir/IKEA-Dataset.git
/content/drive/MyDrive/MTECH_IS_Project/SEM02/temp/ikea_dir/IKEA-Dataset
['.git', 'Baby Children.zip', 'Bathroom.zip', 'Bedroom.zip', 'Dining Room.zip', 'Hallway.zip', 'HomeOffice.zip', 'IKEARequest.py', 'Kitchen 1.zip', 'Kitchen 2.zip', 'Laundry.zip', 'Living Room 1.zip', 'Living Room2.zip', 'README.md']
['/content/drive/MyDrive/MTECH_IS_Project/SEM02/temp/ikea_dir/Baby Children/Baby/Babies Tableware/00319526.jpg', '/content/drive/MyDrive/MTECH_IS_Project/SEM02/temp/ikea_dir/Baby Children/Baby/Babies Tableware/00421411.jpg', '/content/drive/MyDrive/MTECH_IS_Project/SEM02/temp/ikea_dir/Baby Children/Baby/Babies Tableware/20213883.jpg', '/content/drive/MyDrive/MTECH_IS_Project/SEM02/temp/ikea_dir/Baby Children/Baby/Babies Tableware/20421293.jpg', '/content/drive/MyDrive/MTECH_IS_Project/SEM02/temp/ikea_dir/Baby Children/Baby/Babies Tableware

In [141]:
jpg_df = pd.DataFrame(jpg_path_list, columns=['FilePath'])

# Replace backslashes with forward slashes, convert to lowercase, and replace spaces with underscores
jpg_df['FilePath'] = jpg_df['FilePath'].apply(lambda x: x.replace("/", ",").lower().replace(" ", "_"))

# Split the 'FilePath' column into separate columns using a comma as delimiter
jpg_df['Columns'] = jpg_df['FilePath'].str.split(',')

# Determine the maximum number of path parts across all rows
max_path_parts = jpg_df['Columns'].apply(len).max()

# Create separate columns for each path part
for i in range(max_path_parts):
  jpg_df[f'Column_{i+1}'] = jpg_df['Columns'].apply(lambda x: x[i] if len(x) > i else None)

# Drop the intermediate 'PathParts' column
jpg_df = jpg_df.drop(columns=['Columns','FilePath','Column_1','Column_2','Column_3','Column_4','Column_5','Column_6','Column_7'])


jpg_df.head(100)

,Column_8,Column_9,Column_10,Column_11,Column_12,Column_13
0,ikea_dir,baby_children,baby,babies_tableware,00319526.jpg,None
1,ikea_dir,baby_children,baby,babies_tableware,00421411.jpg,None
2,ikea_dir,baby_children,baby,babies_tableware,20213883.jpg,None
3,ikea_dir,baby_children,baby,babies_tableware,20421293.jpg,None
4,ikea_dir,baby_children,baby,babies_tableware,30178006.jpg,None
...,...,...,...,...,...,...
95,ikea_dir,baby_children,baby,changing-table_nursing,10319658.jpg,None
96,ikea_dir,baby_children,baby,changing-table_nursing,10356127.jpg,None
97,ikea_dir,baby_children,baby,changing-table_nursing,20045205.jpg,None
98,ikea_dir,baby_children,baby,changing-table_nursing,30199283.jpg,None


In [142]:
import pandas as pd

columns = ['Column_8', 'Column_9', 'Column_10', 'Column_11', 'Column_12', 'Column_13']

# df = pd.DataFrame(data, columns=columns)

# Create a new DataFrame with the desired output format
output_df = pd.DataFrame(columns=['room', 'class', 'filename'])

for index, row in jpg_df.iterrows():
    room = row['Column_9']
    class_name = row['Column_10']

    # This will check if the value of the row in a particular column has jpg if not it will check the next column
    for column in row.index:
      if 'jpg' in str(row[column]):
        image = row[column]

    # This will add the value to the output df
    if room is not None and class_name is not None and image is not None:
        output_df = output_df.append({'room': room, 'class': class_name, 'filename': image}, ignore_index=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-142-b0a55e54ebbf>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'room': room, 'class': class_name, 'filename': image}, ignore_index=True)
<ipython-input-142-b0a55e54ebbf>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'room': room, 'class': class_name, 'filename': image}, ignore_index=True)
<ipython-input-142-b0a55e54ebbf>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append({'room': room, 'class': class_name, 'filename': image}, ignore_index=True)
<ipython-input-142-b0a55e54ebbf>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas

In [143]:

# Update values in the first column based on conditions

# Update the room column
output_df.loc[output_df['room'] == 'baby_children', 'room'] = 'nursery'
output_df.loc[output_df['room'] == 'dining_room', 'room'] = 'dining room'
output_df.loc[output_df['room'] == 'homeoffice', 'room'] = 'home office'
output_df.loc[output_df['room'].isin(['living_room_1', 'living_room_2']), 'room'] = 'living room'
output_df.loc[output_df['room'].isin(['kitchen_1', 'kitchen_2']), 'room'] = 'kitchen'
output_df.loc[output_df['room'].isin(['living_room_1', 'living_room_2', 'living_room2']), 'room'] = 'living room'

# Update the class column
output_df.loc[output_df['class'] == 'baby', 'class'] = 'baby accessories'
output_df.loc[output_df['class'] == 'children3-7', 'class'] = 'children accesories'
output_df.loc[output_df['class'] == 'bathroomaccessories', 'class'] = 'bathroom accessories'
output_df.loc[output_df['class'] == 'bathroomlighting', 'class'] = 'bathroom lighting'
output_df.loc[output_df['class'] == 'bathroomsinks', 'class'] = 'bathroom sinks'
output_df.loc[output_df['class'] == 'bathroomstorage', 'class'] = 'bathroom storage'
output_df.loc[output_df['class'] == 'showercurtains', 'class'] = 'shower curtains'
output_df.loc[output_df['class'] == 'towels&bathmats', 'class'] = 'towels & bathmats'
output_df.loc[output_df['class'] == 'vanityunits', 'class'] = 'vanity units'
output_df.loc[output_df['class'] == 'openstoragesystem', 'class'] = 'open storage system'
output_df.loc[output_df['class'] == 'wordrobes', 'class'] = 'wardrobes'
output_df.loc[output_df['class'] == 'bathroommirrors', 'class'] = 'bathroom mirrors'

# Replace underscores with space
output_df['class'] = output_df['class'].apply(lambda x: x.replace("_", " "))

output_df.head(10)

,room,class,filename
0,nursery,baby accessories,00319526.jpg
1,nursery,baby accessories,00421411.jpg
2,nursery,baby accessories,20213883.jpg
3,nursery,baby accessories,20421293.jpg
4,nursery,baby accessories,30178006.jpg
5,nursery,baby accessories,30208348.jpg
6,nursery,baby accessories,30421297.jpg
7,nursery,baby accessories,30421301.jpg
8,nursery,baby accessories,40084861.jpg
9,nursery,baby accessories,40199287.jpg


In [144]:
# dataset_loc = '/content/drive/MyDrive/MTECH_IS_Project/SEM02/dataset/path1.csv'
dataset_loc = '/content/drive/MyDrive/MTECH_IS_Documents/PracticeModule/roomifAI/path_ds.csv'

output_df.to_csv(dataset_loc,  encoding='utf-8')

In [145]:
# Merge the output_df to the dataframe with blip2 captioned images

file = '/content/drive/MyDrive/MTECH_IS_Project/SEM02/dataset/ikeads_all_images_blip2.csv'

blip2_df = pd.read_csv(file, index_col=0)  # Replace with your DataFrame source
blip2_df.rename(columns={'filename': 'image'}, inplace=True)
blip2_df['filename'] = blip2_df['image'] + '.jpg'
index_col = "filename"
blip2_df.head(3)


,image,caption,filename
0,30284177,three black metal brackets on a white background,30284177.jpg
1,40336190,red fabric on a white background,40336190.jpg
2,90424599,a wicker outdoor ottoman with a grey cover,90424599.jpg


In [146]:
merge_df = blip2_df.merge(
    output_df, on=index_col, how="left")

merge_df.head(10)

,image,caption,filename,room,class
0,30284177,three black metal brackets on a white background,30284177.jpg,living room,sofas and armchairs
1,40336190,red fabric on a white background,40336190.jpg,living room,sofas and armchairs
2,90424599,a wicker outdoor ottoman with a grey cover,90424599.jpg,living room,sofas and armchairs
3,60406808,a red coffee table with metal legs,60406808.jpg,living room,sofas and armchairs
4,10336243,a close up of a gray fabric,10336243.jpg,living room,sofas and armchairs
5,79261155,an outdoor sectional sofa with black cushions,79261155.jpg,living room,sofas and armchairs
6,89252565,an outdoor sectional sofa with white cushions,89252565.jpg,living room,sofas and armchairs
7,50255536,a coffee cup sitting on top of a wooden tray,50255536.jpg,living room,sofas and armchairs
8,40339829,a yellow checkered sectional couch with a chaise,40339829.jpg,living room,sofas and armchairs
9,99054744,an outdoor sectional sofa with black cushions,99054744.jpg,living room,sofas and armchairs


In [148]:
file = "/content/drive/MyDrive/MTECH_IS_Project/SEM02/dataset/IKEA_SA_Furniture_Web_Scrapings_sss.csv"

scrape_df = pd.read_csv(file, index_col=0)  # Replace with your DataFrame source

scrape_df.rename(columns={'name': 'product_name', 'item_id': 'image',  'short_description': 'notes'}, inplace=True)
scrape_df.drop(scrape_df.columns[[2,4,5,6,7,9]], axis=1, inplace=True)

scrape_df['image'] = scrape_df['image'].astype(str)
index2_col = "image"
scrape_df.head(3)


,image,product_name,price,notes,depth,height,width
0,90420332,FREKVENS,265.0,"Bar table, in/outdoor, 51x51 cm",NaN,99.0,51.0
1,368814,NORDVIKEN,995.0,"Bar table, 140x80 cm",NaN,105.0,80.0
2,9333523,NORDVIKEN / NORDVIKEN,2095.0,Bar table and 4 bar stools,NaN,NaN,NaN


In [149]:
merge2_df = merge_df.merge(
    scrape_df, on=index2_col, how="left")

merge2_df.rename(columns={'image': 'product_id'}, inplace=True)
merge2_df.head(10)

,product_id,caption,filename,room,class,product_name,price,notes,depth,height,width
0,30284177,three black metal brackets on a white background,30284177.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN
1,40336190,red fabric on a white background,40336190.jpg,living room,sofas and armchairs,BRÅTHULT,305.0,Cover for corner sofa,NaN,NaN,NaN
2,90424599,a wicker outdoor ottoman with a grey cover,90424599.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN
3,60406808,a red coffee table with metal legs,60406808.jpg,living room,sofas and armchairs,KROKHOLMEN,175.0,"Coffee table, outdoor",NaN,44.0,NaN
4,10336243,a close up of a gray fabric,10336243.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN
5,79261155,an outdoor sectional sofa with black cushions,79261155.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN
6,89252565,an outdoor sectional sofa with white cushions,89252565.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN
7,50255536,a coffee cup sitting on top of a wooden tray,50255536.jpg,living room,sofas and armchairs,RÖDEBY,69.0,Armrest tray,37.0,NaN,65.0
8,40339829,a yellow checkered sectional couch with a chaise,40339829.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN
9,99054744,an outdoor sectional sofa with black cushions,99054744.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
# load the SG Dataset
file = "/content/drive/MyDrive/MTECH_IS_Project/SEM02/dataset/furniture_tags.xlsx"
fur_tags_df = pd.read_excel(file)
fur_tags_df.drop(fur_tags_df.columns[[6,7,8,9,10,11]], axis=1, inplace=True)
# index3_col = 'filename'
# index3_col = ['filename', 'caption', 'product_id', 'room', 'class', 'product_name']
index3_col = ['filename']
fur_tags_df['product_id'] = fur_tags_df['product_id'].astype(str)
fur_tags_df.head(100)

,filename,product_id,product_name,room,class,caption
0,00121766.jpg,121766.0,NaN,kitchen,bar stool,"a photo of white, wooden tall bar stool with b..."
1,00217801.jpg,217801.0,NaN,kitchen,bar stool,"a photo of brown, wooden medium height bar sto..."
2,00272656.jpg,272656.0,NaN,kitchen,bar stool,"a photo of modern industrial style white, wood..."
3,00400550.jpg,400550.0,NaN,kitchen,bar stool,"a photo of classical style brown, wooden tall ..."
4,10122647.jpg,10122647.0,NORDVIKEN / NORDVIKEN,kitchen,bar stool,"a photo of white, wooden tall bar stool with b..."
...,...,...,...,...,...,...
95,70340807.jpg,70340807.0,KLIMPEN / TORSKLINT,dining,dining table,"a photo of white, four-legged, wooden, medium ..."
96,70340812.jpg,70340812.0,KLIMPEN / TORSKLINT,dining,dining table,"a photo of chestnut brown, four-legged, wooden..."
97,70340826.jpg,70340826.0,KLIMPEN / TORSKLINT,dining,dining table,"a photo of white, four-legged, wooden, medium ..."
98,70359991.jpg,70359991.0,KLIMPEN / TORSKLINT,dining,dining table,"a photo of black, wooden, four-legged, long re..."


In [158]:
# Combine values of two columns with a comma and create a new column
merge2_df['Combined_Columns'] = merge2_df['caption'] + ', ' + fur_tags_df['caption']

merge4_df = merge2_df.merge(
    fur_tags_df, on=index3_col, how="left")

# merge4_df.drop(merge4_df.columns[[2,7,12,13,14,15,16]], axis=1, inplace=True)

merge4_df.head(3)

,product_id_x,caption_x,filename,room_x,class_x,product_name_x,price,notes,depth,height,width,Combined_Columns,product_id_y,product_name_y,room_y,class_y,caption_y
0,30284177,three black metal brackets on a white background,30284177.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN,three black metal brackets on a white backgrou...,NaN,NaN,NaN,NaN,NaN
1,40336190,red fabric on a white background,40336190.jpg,living room,sofas and armchairs,BRÅTHULT,305.0,Cover for corner sofa,NaN,NaN,NaN,"red fabric on a white background, a photo of b...",NaN,NaN,NaN,NaN,NaN
2,90424599,a wicker outdoor ottoman with a grey cover,90424599.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,NaN,"a wicker outdoor ottoman with a grey cover, a ...",NaN,NaN,NaN,NaN,NaN


In [159]:
merge4_df.drop(merge4_df.columns[[1,6,12,13,14,15,16]], axis=1, inplace=True)



merge4_df.rename(columns={'product_id_x': 'product_id', 'room_x': 'room',  'class_x': 'class',  'product_name_x': 'product_name', 'Combined_Columns': 'caption'}, inplace=True)

merge4_df.head(3)

,product_id,filename,room,class,product_name,notes,depth,height,width,caption
0,30284177,30284177.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,three black metal brackets on a white backgrou...
1,40336190,40336190.jpg,living room,sofas and armchairs,BRÅTHULT,Cover for corner sofa,NaN,NaN,NaN,"red fabric on a white background, a photo of b..."
2,90424599,90424599.jpg,living room,sofas and armchairs,NaN,NaN,NaN,NaN,NaN,"a wicker outdoor ottoman with a grey cover, a ..."


In [160]:
output_file = "/content/drive/MyDrive/MTECH_IS_Project/SEM02/dataset/ikea_furniture_tags.csv"

merge4_df.to_csv(output_file,  encoding='utf-8')

In [161]:
# Close the logging handler
logging.shutdown()